# 0. Setup

In [ ]:
!nvidia-smi -L

In [1]:
import os
import datetime
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd


from model.vit_for_small_dataset import ViT
from utils.imageset_handler import ImageQualityDataset

In [ ]:
def find_pth_files(directory_path):
    """
    Find and return a list of full paths to .pth files in the specified directory.

    Args:
        directory_path (str): The directory path to search for .pth files.

    Returns:
        List[str]: A list of full paths to .pth files.
    """
    pth_files = []
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith(".pth"):
                pth_files.append(os.path.join(root, file))
    return pth_files

# 1. Build Model

### 1.1 Define Variables

In [45]:
image_size=256
patch_size=16
num_classes=5  # Number of classes for image quality levels
dim=1024
depth=6
heads=16
mlp_dim=2048
emb_dropout=0.1

### 1.2 Compile

In [ ]:
model = ViT(
    image_size=image_size,
    patch_size=patch_size,
    num_classes=num_classes,
    dim=dim,
    depth=depth,
    heads=heads,
    mlp_dim=mlp_dim,
    emb_dropout=emb_dropout
)
print(model)

ViT(
  (to_patch_embedding): SPT(
    (to_patch_tokens): Sequential(
      (0): Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=16, p2=16)
      (1): LayerNorm((3840,), eps=1e-05, elementwise_affine=True)
      (2): Linear(in_features=3840, out_features=1024, bias=True)
    )
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (transformer): Transformer(
    (layers): ModuleList(
      (0-5): 6 x ModuleList(
        (0): PreNorm(
          (norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fn): LSA(
            (attend): Softmax(dim=-1)
            (dropout): Dropout(p=0.0, inplace=False)
            (to_qkv): Linear(in_features=1024, out_features=3072, bias=False)
            (to_out): Sequential(
              (0): Linear(in_features=1024, out_features=1024, bias=True)
              (1): Dropout(p=0.0, inplace=False)
            )
          )
        )
        (1): PreNorm(
          (norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (

# 2 Load Dataset

In [ ]:
#weights_path = f'{results_path}/vit_model_20230821_121731_epoch_2of20_valLoss_1.572_valAcc_0.267_batchsize_64_lr_0.0_TestImg.pth'
weights_dir = '/home/maxgan/WORKSPACE/UNI/BA/vision-transformer-for-image-quality-perception-of-individual-observers/results/weights/all_distored_imgs'
# weights_path = f'/home/maxgan/WORKSPACE/UNI/BA/vision-transformer-for-image-quality-perception-of-individual-observers/results/weights/TEST/vit_model_20230821_120855_epoch_16of20_valLoss_7.457_valAcc_0.233_batchsize_64_lr_0.0_TestImg.pth'

csv_file = '/home/maxgan/WORKSPACE/UNI/BA/vision-transformer-for-image-quality-perception-of-individual-observers/assets/Test/AccTestCsv/objectiveAccTest.csv'
dataset_root =  '/home/maxgan/WORKSPACE/UNI/BA/vision-transformer-for-image-quality-perception-of-individual-observers/assets/Test/TestImg'
batch_size = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### 2.1 Add Augmentation (Transformation)

In [47]:
transform = transforms.Compose([
    transforms.RandomResizedCrop(256),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet mean and std
])

### 2.2 Loading

In [ ]:
# Initialize your dataset loader and test dataset
test_dataset = ImageQualityDataset(csv_file,dataset_root, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Number of images in the dataset: 300


# 3. Evaluate

### 3.1 Evaluating best weights by calculating class probability, MSE of most likely class and weighted sum, mean entropy, accuracy and classification report

In [102]:
import torch
import torch.nn as nn
from sklearn.metrics import mean_squared_error, classification_report, confusion_matrix
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

# List of different weight files
weight_files = find_pth_files(weights_dir)

results = []
example_pred_results = []

for weight_file in weight_files:
    print(f'Weights-file: {os.path.basename(weight_file)} will be evaluated')
    # Load the model with different weights
    model.load_state_dict(torch.load(weight_file))
    model.eval()

    true_labels = []
    test_preds = []
    entropies = []
    weighted_sums = []
    with torch.no_grad():

        for i, (images, labels) in enumerate(test_loader, 0):
            # images = images.to(device)
            # labels = labels.to(device)
            print(f"Example Prediction of Batch: {i}")
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            test_preds.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())


            # Convert logits to probabilities
            probabilities = nn.functional.softmax(outputs, dim=1)
            # Format probabilities in a readble way
            formatted_probs = [[f'{p:.4f}' for p in prob_list] for prob_list in probabilities.numpy()]

            # Calculate Entropy
            entropy_values = -torch.sum(probabilities * torch.log2(probabilities + 1e-10), dim=1).cpu().numpy()
            # Format entropies in a readble way
            formatted_entropy = [f'{entropy:.4f}' for entropy in entropy_values]
            entropies.extend(entropy_values)
            
            # Define weighting factors
            weighting_factors = [0,1,2,3,4]
            # Calculate the weighted sum of probabilities
            weighted_sum = torch.sum(probabilities * torch.tensor(weighting_factors), dim=1).cpu().numpy()
            # Format weighted sum in a readble way
            formatted_weighted_sum = [f'{sum:.4f}' for sum in weighted_sum]
            weighted_sums.extend(weighted_sum)

                        # Example printout for the first batch (you can customize as needed)
            if i == 0:
                example_pred_result = {
                    "Weights File": os.path.basename(weight_file),
                    "True Label": labels.cpu().numpy()[0],
                    "Predicted Label": preds.cpu().numpy()[0],
                    "Weighted Sum of Probability": formatted_weighted_sum[0],
                    "Predicted Class Probability": formatted_probs[0],
                    "Entropy Value": formatted_entropy[0],
                }
                example_pred_results.append(example_pred_result)
            print(f'True-Label: {labels.cpu().numpy()[0]}')
            print(f'Predicted-Label: {preds.cpu().numpy()[0]}')
            print(f'Weighted Sum of Probability: {formatted_weighted_sum[0]}')  # Gewichtete Summe der Wahrscheinlichkeiten
            print(f'Predicted-Class-Probality: {formatted_probs[0]}')
            print(f'Entropy Value: {formatted_entropy[0]}\n') # High Value: spreading; Low Value: concentrated



    # Calculate the MSE of weighted sum and ground truth
    mse_weighted = mean_squared_error(true_labels, weighted_sums)

    # Calculate the MSE of most likely class and ground truth
    mse = mean_squared_error(true_labels, test_preds)
    
    # Calculate the Mean Entropy
    mean_entropy = np.mean(entropies)
    # Calculate Accuracy
    accuracy = accuracy_score(true_labels, test_preds)

    # Generate classification report
    class_report = classification_report(true_labels, test_preds)

    # Generate confusion matrix
    confusion = confusion_matrix(true_labels, test_preds)
    print('Model Summary:')
    print(f'Weight: {os.path.basename(weight_file)}, Accuracy: {accuracy}, Mean Entropy: {mean_entropy}\nClassification Report:\n{class_report}')
    # Save confusion matrix as a figure
    plt.figure(figsize=(8, 6))
    sns.heatmap(confusion, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.savefig(weight_file.replace(".pth", "_confusion.png"))
    plt.close()

    # Store the results
    results.append({
        "Weights File": os.path.basename(weight_file),
        "Accuracy": accuracy,
        "MSE": mse,
        "MSE weighted": mse_weighted,
        "Mean Entropy": mean_entropy,
        "Classification Report": class_report
    })

# Create a DataFrame and save to CSV
import pandas as pd
results_df = pd.DataFrame(results)
results_path = os.path.join(weights_dir, "model_comparison_results.csv")
results_df.to_csv(results_path, index=False)

# Save example printouts to a CSV file for this model
example_printouts_df = pd.DataFrame(example_pred_results)
example_printout_file = os.path.join(weights_dir, "model_comparison_results_examples.csv")
example_printouts_df.to_csv(example_printout_file, index=False)

Weights-file: vit_model_20230822_000007_epoch_8of50_valLoss_0.374_valAcc_0.836_batchsize_128_lr_0.0_allDistorted.pth will be evaluated
Example Prediction of Batch: 0
True-Label: 2
Predicted-Label: 2
Weighted Sum of Probability: 2.2668
Predicted-Class-Probality: ['0.0049', '0.0181', '0.6830', '0.2936', '0.0005']
Entropy Value: 1.0427

Example Prediction of Batch: 1
True-Label: 4
Predicted-Label: 4
Weighted Sum of Probability: 3.9993
Predicted-Class-Probality: ['0.0000', '0.0000', '0.0000', '0.0007', '0.9993']
Entropy Value: 0.0079

Example Prediction of Batch: 2
True-Label: 1
Predicted-Label: 1
Weighted Sum of Probability: 0.9367
Predicted-Class-Probality: ['0.0683', '0.9274', '0.0038', '0.0005', '0.0000']
Entropy Value: 0.4019

Example Prediction of Batch: 3
True-Label: 4
Predicted-Label: 4
Weighted Sum of Probability: 3.9931
Predicted-Class-Probality: ['0.0000', '0.0000', '0.0000', '0.0069', '0.9931']
Entropy Value: 0.0597

Example Prediction of Batch: 4
True-Label: 3
Predicted-Label:

### 3.2 Plot Distribution

In [ ]:
csv_file = "/home/maxgan/WORKSPACE/UNI/BA/TIQ/assets/Test/AccTestCsv/shinyxAccTest20-01-2023.csv"
output_image_path = "/home/maxgan/WORKSPACE/UNI/BA/TIQ/assets/Test/AccTestCsv/rating_distribution_shinyxAccTest.png"

In [ ]:
data = pd.read_csv(csv_file, header=None, skiprows=1)

# Map rating values to their corresponding labels
rating_labels = {
    1: "Bad",
    2: "Insufficient",
    3: "Fair",
    4: "Good",
    5: "Excellent"
}
data["Rating_Label"] = data[1].map(rating_labels)

# Group data by Rating_Label and count occurrences
class_counts = data["Rating_Label"].value_counts().sort_index()
# Calculate total number of images
total_images = class_counts.sum()

In [ ]:
# Create a bar chart
plt.figure(figsize=(10, 6))
class_counts.plot(kind="bar", color='skyblue')
plt.title("Image Rating Distribution Person1 (shinyx)")
plt.xlabel("Rating")
plt.ylabel("Number of Images")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(output_image_path)
plt.show()
# Display the table
print("Rating Distribution Table:")
print(class_counts)